In [ ]:
import pandas as pd
import numpy as np
from datetime import date, time, datetime, timedelta

import plotly.graph_objects as go

import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter,ScalarFormatter

In [ ]:
PATH_EXCEL = r'D:\WORKS\COMSOL_polymers\tables'
STD_VALUES = {
        'k1': 1E+9,
        'k2': 1E+9,
        'k3': 1E+9,
        'k4': 1E+9,
        'k5': 2000,
        'k6': 2,
        'k7': 0.05,
        'k8': 1,
        'k9': 0.001,
        'k10': 1E+9,
        'k11': 1E-5,
    }

In [ ]:
def df_from_comsol(
    name,
    from_name=r'\out.csv',
    temp_name=r'\temp.csv',
):
    with open(PATH_EXCEL + from_name, 'r') as file:
        lines = file.readlines()

    with open(PATH_EXCEL + temp_name, 'w') as file:
        for line in lines[4:]:
            file.write(line)
    df = pd.read_csv(PATH_EXCEL + r'\temp.csv')
    df = df.rename(
        columns={
            df.columns[0]: df.columns[0].replace('% ', ''),
            'Time=1E-4, timeint(0, 1E-4, D)': 'target',
        })
    df['name'] = name
    df['date'] = datetime.now().replace(second=0, microsecond=0)
    return df


def add_notes(from_name=r'\out.csv'):
    name = ''
    while True:
        name = input('Name (q for quit): ')
        if (name != '') and (name != 'q'):
            try:
                old_df = pd.read_excel(PATH_EXCEL + r'\base.xlsx', index_col=0)
            except FileNotFoundError:
                old_df = pd.DataFrame(
                    {key: [None]
                     for key in STD_VALUES.keys()})
                old_df = old_df.drop(index=0)

            new_df = pd.concat(
                [old_df, df_from_comsol(name, from_name=from_name)],
                ignore_index=True,
            )
            new_df = new_df.fillna(STD_VALUES)
            new_df.to_excel(PATH_EXCEL + r'\base.xlsx')

            return new_df, old_df
        elif name == 'q':
            break


def get_const_diap(df, print_only=False):
    useless = [
        'light',
        'target',
        'name',
        'date',
    ]
    values = {}
    for i in df:
        if i in useless: continue
        values.update({i: df[i].unique()})
        if print_only: print(f'{i:<5}:{values[i]}')

    if not print_only: return values


In [ ]:
add_notes()